In [ ]:
import pandas as pd

#df_books = pd.read_excel("Skripsi/Dataset rapih 4sheet_skripsi_revisi.xlsx",sheet_name="Gabungan")

df_books = pd.read_excel("/content/drive/MyDrive/Folder Skripsi CF_buku/Dataset rapih 4sheet_skripsi_revisi_cuma 1 sheet.xlsx",
                         sheet_name="15tahun")
df_books.dropna(axis=0, how="any")
df_user = pd.read_excel("/content/drive/MyDrive/Folder Skripsi CF_buku/User_skripsi.xlsx", sheet_name="GabunganU")#GabunganU, sheet sampe 7
df_user['Rating'] = df_user['Rating'].replace(['it was amazing', 'really liked it', "liked it",\
                                               "did not like it","it was ok","This user doesn't have any rating"],\
                                              [5, 4, 3,1,2,0])
df_user = df_user.dropna(axis=0, how='any')

df_books["Name"] = df_books["Name"].astype(str)
df_user["Name"] = df_user["Name"].astype(str)

df_user.drop(df_user.index[df_user['Rating'] == 0], inplace=True)

df_books['RatingDistTotal'] = df_books['RatingDistTotal'].\
astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

df_books_eval_replace = df_books[["RatingDist1", "RatingDist2", "RatingDist3", "RatingDist4", "RatingDist5"]]
list_nama_column_eval_replace = ["RatingDist1", "RatingDist2", "RatingDist3", "RatingDist4", "RatingDist5"]

for i in range(5):
    df_books_eval_replace[
        list_nama_column_eval_replace[i]] = df_books_eval_replace[
            list_nama_column_eval_replace[i]].astype(str)
    df_books_eval_replace[
        list_nama_column_eval_replace[i]] = df_books_eval_replace[
            list_nama_column_eval_replace[i]].str.replace(':','/')

for i in range(5):
    substitute_column = []
    for j in range(len(df_books_eval_replace)):
        if type(df_books_eval_replace.iloc[:, i][j]) == str:
            substitute_column.append(eval(df_books_eval_replace.iloc[:, i][j]))
        else:
            substitute_column.append(df_books_eval_replace.iloc[:, i].loc[j])
    df_books[list_nama_column_eval_replace[i]] = substitute_column

df_books


In [ ]:
df_merge_inner=df_user.merge(df_books, left_on='Name', right_on='Name', how='inner')

df_merge_inner_pivot=df_merge_inner.pivot_table(index='ID', columns='Name', values='Rating_x')
df_merge_inner_pivot

# HDBSCAN-RSVD-CF

In [ ]:
pip install hdbscan

In [ ]:
#vektor adalah perwakilan titik
import hdbscan

clusterer_user = hdbscan.HDBSCAN(min_cluster_size=2,min_samples=1,gen_min_span_tree=True)

clusterer_user.fit(df_merge_inner_pivot.fillna(0))

In [ ]:
# Sesi Cross Val
import math
urutan_list_bagi_5 = []
for i in range(5):
    urutan_list_bagi_5.append(
            df_merge_inner_pivot.iloc[math.ceil((i)*20 / 100 *
                                                len(df_merge_inner_pivot)):(i+1) *
                                      math.ceil(20 / 100 *
                                                len(df_merge_inner_pivot)), :])

In [ ]:
df_merge_inner_pivot["user_cluster"]=clusterer_user.labels_
df_merge_inner_pivot=df_merge_inner_pivot.dropna(axis=1,how='all')
df_merge_inner_pivot

In [ ]:
#Revisi ke-3 7-6-2023
#modifikasinya buat split test
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from sklearn.utils.extmath import randomized_svd
import math

#inisiasi
start_time = time.time()

mean_dari_mean_rmse_kuintil=[]
panjang_kumpulan_rmse_kuintil=[]
kumpulan_dari_kumpulan_waktu_kuintil=[]

list_matrixU_rsvd = []

#Proses RSVD + Similarity
#jika di cluster lebih dari 10 orang, maka dilakukan pemotongan
for poke in range(df_merge_inner_pivot["user_cluster"].max() + 2):
    if len(df_merge_inner_pivot.groupby("user_cluster").get_group(poke -
                                                                  1)) >= 10:
        U, s, Vh = randomized_svd(
            df_merge_inner_pivot.groupby("user_cluster").get_group(
                poke - 1).drop(columns='user_cluster').fillna(0),
            n_components=math.ceil(
                len(
                    df_merge_inner_pivot.groupby("user_cluster").get_group(
                        poke - 1).drop(columns='user_cluster').fillna(0)) *
                80 / 100),random_state=0)   #80 bisa diganti sesuai keinginan pengambilan n-component

        df_pivot_avg_rsvd_U = pd.DataFrame(U)
        df_pivot_avg_rsvd_U = df_pivot_avg_rsvd_U.set_index(
            df_merge_inner_pivot.groupby("user_cluster").get_group(
                poke - 1).drop(columns='user_cluster').fillna(0).index)
        df_pivot_avg_rsvd_U = df_pivot_avg_rsvd_U.T.corr()
        list_matrixU_rsvd.append(df_pivot_avg_rsvd_U)

    else:
        U, s, Vh = randomized_svd(
            df_merge_inner_pivot.groupby("user_cluster").get_group(poke-1).drop(
                columns='user_cluster').fillna(0),
            n_components=len(df_merge_inner_pivot.groupby("user_cluster").\
                             get_group(poke-1).drop(columns='user_cluster')),random_state=0)

        df_pivot_avg_rsvd_U = pd.DataFrame(U)
        df_pivot_avg_rsvd_U = df_pivot_avg_rsvd_U.set_index(
            df_merge_inner_pivot.groupby("user_cluster").get_group(
                poke - 1).drop(columns='user_cluster').fillna(0).index)
        df_pivot_avg_rsvd_U = df_pivot_avg_rsvd_U.T.corr()
        list_matrixU_rsvd.append(df_pivot_avg_rsvd_U)
print("--- %s seconds ---" % (time.time() - start_time))

w_awal=(time.time() - start_time)

#Prediksi rating buku oleh pengguna pada model HDBSCAN-RSVD-CF
for kuintil in range(5):
    picked_userid = urutan_list_bagi_5[kuintil].index
    kumpulan_rmse = []
    for i in range(len(picked_userid))
        user_active_cluster = df_merge_inner_pivot.loc[picked_userid[i]]["user_cluster"]
        user_similarity = list_matrixU_rsvd[int(user_active_cluster)+1]

        # no 1
        #Kegunaan T disini supaya dia transpose dan si Index itu yang kita Pearson

        #========================================================================
        # Variabel hapus pengguna aktif (data testing)
        remove_user=user_similarity[user_similarity.index.isin(picked_userid)].index
        # Hapus data testing supaya pemilihan korelasi Pearson tidak muncul dirinya sendiri
        user_similarity = user_similarity.drop(index=remove_user)
        #========================================================================
        # Parameter batas similarity
        user_similarity_threshold = 0.4
        # mencari orang-orang yang memiliki nilai similarity >= 0.4
        similar_users = user_similarity[
            user_similarity[picked_userid[i]] > user_similarity_threshold][
                picked_userid[i]].sort_values(ascending=False)
        # Print daftar pengguna yang mirip dengna pengguna aktif
        print(f'Proses n=10:\nThe similar users for user {picked_userid[i]} are',
              similar_users)

        #========================================================================
        # no 2
        # Buku buku yang sudah dibaca oleh pengguna aktif
        picked_userid_watched = df_merge_inner_pivot.\
        drop(columns="user_cluster")[df_merge_inner_pivot.drop(columns="user_cluster").\
                                     index == picked_userid[i]].dropna(axis=1, how='all')

        # Menyesuaikan buku pengguna tak aktif dengan pengguna aktif
        similar_user_buku = df_merge_inner_pivot.\
        drop(columns="user_cluster")[df_merge_inner_pivot.drop(columns="user_cluster").\
                                     index.isin(similar_users.index)].dropna(axis=1, how='all')
        #========================================================================

        #3
        # membangun prediksi rating buku [sum (similarity * selisih rating pengguna tak aktif)/sum similiartiy] Perhatikan paper Geetha/skripsi
        item_score = {}
        # Looping buku
        for j in similar_user_buku.columns:
            # rating dari 1 buku (nanti iterasi)
            buku_rating = similar_user_buku[j]
            # total sebagai vairaibel dari prediksi bagian (similarity * selisih rating pengguna tak aktif)
            total = 0
            #tambahin mean di sini, nanti tinggal kurangin deh untuk si selisih rating pengguna tak aktif
            mean_rating_user_u = buku_rating.mean()

            # Proses looping pembentukan [sum (similarity * selisih rating pengguna tak aktif)/sum similiartiy] dari tiap pengguna serupa
            for u in similar_users.index:
                # fungsi if untuk mendanakan kalau si buku punya rating/tidak kosong
                if pd.isna(buku_rating[u]) == False:
                    # nilai dari (similarity * selisih rating pengguna tak aktif)
                    score = similar_users[u] * abs(buku_rating[u] -
                                                   mean_rating_user_u)
                    # tambahin terus totalnya, karena sum
                    total += score

            # ini hasil akhir dari [sum (similarity * selisih rating pengguna tak aktif)/sum similiartiy] tetapi untuk 1 buku
            item_score[j] = total / sum(similar_users)
        # konversi dictionary jadi pandas df
        item_score = pd.DataFrame(item_score.items(),
                                  columns=['books', 'books_score'])

        # sorting niali buku
        ranked_item_score = item_score.sort_values(by='books_score',
                                                   ascending=False)
        #========================================================================
        #4
        # bagian rata-rata penialain buku oleh pengguna aktif dari prediksi rating buku
        avg_rating = picked_userid_watched[picked_userid_watched.index ==
                                           picked_userid[i]].T.mean()[
                                               picked_userid[i]]
        # print average penilaian buku si pengguna aktif
        print(
            f'The average books rating for user {picked_userid[i]} is {avg_rating:.2f}'
        )

        # Kalkulasi prediksi rating bukunya berdasarkan hasil yang didapat di proses-proses atas
        ranked_item_score_plus_avg_rating = []

        for k in range(len(ranked_item_score)):
            if (ranked_item_score['books_score'][k] + avg_rating) >= 5:
                ranked_item_score_plus_avg_rating.append(5)
            else:
                ranked_item_score_plus_avg_rating.append(
                    ranked_item_score['books_score'][k] + avg_rating)

        ranked_item_score['predicted_rating'] = ranked_item_score_plus_avg_rating

        ranked_item_score.sort_values(by=["predicted_rating"],
                                      inplace=True,
                                      ascending=False)
        #========================================================================
        #RMSE
        #Kasus Geetha--sederhana
        user_active_picked = picked_userid_watched.T

        CF_model = ranked_item_score[ranked_item_score["books"].isin(
            user_active_picked.index[:])]
        CF_model = CF_model.sort_values(by=['books'])


        user_active_picked = user_active_picked[user_active_picked.T.columns.isin(
            CF_model["books"])]
        user_active_picked = user_active_picked.sort_index()

        #Hasil Geetha, user active

        if len(user_active_picked.iloc[:, 0]) != 0 and len(
                CF_model["predicted_rating"]) != 0:
            rms = sqrt(
                mean_squared_error(user_active_picked.iloc[:, 0],
                                   CF_model["predicted_rating"]))
            kumpulan_rmse.append(rms)
            print(f"Nilai RMSEnya adalah:\t{rms}\n")
        print(f"ini nilai i: {i}")

    mean_dari_mean_rmse_kuintil.append(np.mean(kumpulan_rmse))
    panjang_kumpulan_rmse_kuintil.append(len(kumpulan_rmse))
    kumpulan_dari_kumpulan_waktu_kuintil.append((time.time() - start_time)-w_awal)

    print("--- %s seconds ---" % (kumpulan_dari_kumpulan_waktu_kuintil[kuintil]))
    print(f"ini adalah meannya dan len-nya:\t{mean_dari_mean_rmse_kuintil[kuintil]};\t{panjang_kumpulan_rmse_kuintil[kuintil]}")


In [ ]:
print(w_awal)
print(mean_dari_mean_rmse_kuintil)
print(panjang_kumpulan_rmse_kuintil)
print(kumpulan_dari_kumpulan_waktu_kuintil)
